## ibss to goms convert (지급율)

In [ ]:
import pandas as pd
import GladUtil as gd

COMP_LONG = {'흥국화재':'흥국','한화손보':'한화','현대해상':'현대','삼성화재':'삼성','농협손보':'농협','DB손보':'DB','KB손보':'KB','메리츠손보':'메리츠',
             '롯데손보':'롯데','MG손보':'MG','AIG손해보험':'AIG','처브손해보험':'처브','하나손해보험':'하나','DAS':'DAS','메리츠화재':'메리츠'}
DICT_JONG = {'보장성':'보장','재물성':'재물','연금성':'연금','저축성':'저축','실손':'실손'}

new_comp = dict([(value, key) for key, value in COMP_LONG.items()])
new_jong = dict([(value, key) for key, value in DICT_JONG.items()])


In [ ]:
path = 'E:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/4.수수료Table(Upload)/ibss업로드/'
in_file = '장기_2022-02 손보수수료(GLD-900)_202202.xlsx'

sheet = 'Worksheet'
c = '회사 보종 실손시작회 기간1-시작 기간1-끝 기간1-요율 기간2-시작 기간2-끝 기간2-요율 기간3-시작 기간3-끝 기간3-요율 기간4-시작 기간4-끝 기간4-요율 기간5-시작 기간5-끝 기간5-요율 기간6-시작 기간6-끝 기간6-요율 기간7-시작 기간7-끝 기간7-요율 기간8-시작 기간8-끝 기간8-요율 기간9-시작 기간9-끝 기간9-요율 기간10-시작 기간10-끝 기간10-요율'
columns = c.split(' ')
columns = columns[:24]

df = pd.read_excel(path+in_file,sheet_name=sheet,header=2,names=columns)
df_temp = pd.read_excel(path+in_file,sheet_name=sheet)
rule_nm = df_temp.iloc[0][0]
df['정산구분'] = '원수사성적'
df['정산패턴명'] = rule_nm
df['상품군분류'] = '소분류'
df['회사'] = df['회사'].apply(lambda x: gd.convertByVal(x,new_comp))
df['보종'] = df['보종'].apply(lambda x: gd.convertByVal(x,new_jong))
df.sort_values(by=['회사','보종'],ascending=[True,True],inplace=True)
df = df[df.회사 != 'DAS']

columns.insert(0,"정산구분")
columns.insert(1,"정산패턴명")
columns.insert(3,"상품군분류")
columns.remove("실손시작회")

df_unload = df[columns]
df_unload = df_unload.fillna("")
df_unload = df_unload[df_unload['기간1-요율']!='']
#--빈행 1개 삽입
import numpy as np
dt = pd.DataFrame([[np.nan] * len(df_unload.columns)], columns=df_unload.columns)
dt = dt.append(df_unload, ignore_index=True)

in_file = in_file.replace("IBSS","GOMS")
outFile = f'E:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/4.수수료Table(Upload)/goms업로드/{in_file}' 
with pd.ExcelWriter(outFile) as writer:  
    dt.to_excel(writer, sheet_name='규정생성관리',index=False)

## ibss to goms (환수규정)

In [38]:
COMP_LIFE = {'흥국생명':'흥국','한화생명':'한화','푸르덴셜생명':'푸르','카디프생명':'카디프','처브라이프':'처브','오렌지라이프':'오렌지','신한생명':'신한','삼성생명':'삼성',
             '미래에셋생명':'미래','메트라이프생명':'메트','라이나생명':'라이나','동양생명':'동양','농협생명':'농협','교보생명':'교보','KDB생명':'kdb','KB생명':'KB','IBK연금보험':'IBK',
             'DGB생명':'DGB','DB생명':'DB','ABL생명':'ABL','AIA생명':'AIA','푸본현대생명':'현대','PCA생명':'PCA'}
new_comp_li = dict([(value, key) for key, value in COMP_LIFE.items()])

In [41]:
path ='E:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/4.수수료Table(Upload)/환수율/'
in_file = '생명환수_2022-02 생보수수료 환수율_202202.xlsx'
보종 = '생보'

sheet = 'Worksheet'
c = '보험사 정산타입 1회 2회 3회 4회 5회 6회 7회 8회 9회 10회 11회 12회 13회 14회 15회 16회 17회 18회 19회 20회 21회 22회 23회 24회'
columns = c.split(' ')

df = pd.read_excel(path+in_file,sheet_name=sheet,header=2,names=columns)
df_temp = pd.read_excel(path+in_file,sheet_name=sheet)
rule_nm = df_temp.iloc[0][0]

df.rename(columns={"보험사":"회사"}, inplace=True)
df['규정명'] = rule_nm
df['상품군'] = '전체'
df['상품군분류'] = '전체'
df['상태'] = '해지'
if 보종 == '손보':
    df['회사'] = df['회사'].apply(lambda x: gd.convertByVal(x,new_comp))
else:
    df['회사'] = df['회사'].apply(lambda x: gd.convertByVal(x,new_comp_li))
df = df[~df['회사'].isin(['DAS','기타'])]

# df['보종'] = df['보종'].apply(lambda x: gd.convertByVal(x,new_jong))
# df.sort_values(by=['회사','보종'],ascending=[True,True],inplace=True)

columns.insert(0,"규정명")
columns.insert(2,"상품군분류")
columns.insert(3,"상품군")
columns.insert(4,"상태")
columns[1] = "회사"
columns.remove("정산타입")

df_unload = df[columns]
df_unload = df_unload.fillna("")
df_unload.loc[df_unload.회사=="기본적용", "회사"]="기본"
for i in range(25,37):
    df_unload[ str(i) + '회' ] = ""
df_unload[ '37회이후' ] = ""    
    
df_unload = df_unload.drop_duplicates()
# 1 column shift to left
# df_unload = pd.concat([df_unload.iloc[:,:5],df_unload.iloc[:,5:].shift(periods=-1, axis="columns").fillna("")],axis=1)

# 기본 100율 생성 (1~37회) 및 상태별 row생성
df_temp = df_unload[df_unload['회사']=='기본'].copy()
df_temp.iloc[:,5:] = 100
df_temp["상태"] = "부활"
df_temp2 = pd.concat([df_temp, df_temp.replace('부활','취소'),df_temp.replace('부활','철회'),df_temp.replace('부활','무효'),
                      df_temp.replace('부활','품질보증'),df_temp.replace('부활','민원해지'),df_temp.replace('부활','승환')])

df_unload = pd.concat([df_unload, df_unload.replace('해지','해약'),df_unload.replace('해지','실효'),df_unload.replace('해지','감액')])
    
df_unload = pd.concat([df_unload, df_temp2])
df_unload.sort_values(by=['회사','상태'],ascending=[True,True],inplace=True)
# 생보는 해지일 경우 모든 회차100%환수
if 보종 == '생보':
    df_unload.loc[df_unload['상태']=='해지',df_unload.columns[5:]] = 100

outFile = f'E:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/4.수수료Table(Upload)/환수율/GOMS_{in_file}'

with pd.ExcelWriter(outFile) as writer:  
    df_unload.to_excel(writer, sheet_name='규정생성관리',index=False)

## ibss to goms (자동차)

In [ ]:
path ='E:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/4.수수료Table(Upload)/ibss업로드/'
in_file = '자동차_2021-09 자동차수수료(표준FC_11%)_202109.xlsx'
sheet = 'Worksheet'
c = '상품 삼성화재 현대해상 KB손보 메리츠화재 DB손보 한화손보 롯데손보 흥국화재 MG손보 농협손보 DAS AIG손해보험 처브손해보험 하나손해보험 기타'
columns = c.split(' ')

df = pd.read_excel(path+in_file,sheet_name=sheet,header=2,names=columns)
df_temp = pd.read_excel(path+in_file,sheet_name=sheet)
rule_nm = df_temp.iloc[0][0]

df['규정명'] = rule_nm
df['정산구분'] = '자동차보험료'
df['구분'] = '자동차'
df = df[df.상품 != '이륜용도']

columns.insert(0,"정산구분")
columns.insert(1,"규정명")
columns.insert(2,"구분")
columns.remove("DAS")
columns.remove("기타")

df_unload = df[columns]
df_unload = df_unload.fillna('')

outFile = f'E:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/4.수수료Table(Upload)/goms업로드/GOMS_{in_file}' 
with pd.ExcelWriter(outFile) as writer:  
    df_unload.to_excel(writer, sheet_name='규정생성관리',index=False)